<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Importing-Raw-Data" data-toc-modified-id="Importing-Raw-Data-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Importing Raw Data</a></span></li><li><span><a href="#Importing-enriched-data" data-toc-modified-id="Importing-enriched-data-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Importing enriched data</a></span></li><li><span><a href="#Spatial-Join" data-toc-modified-id="Spatial-Join-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Spatial Join</a></span></li><li><span><a href="#Aggregation" data-toc-modified-id="Aggregation-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Aggregation</a></span></li><li><span><a href="#Plots" data-toc-modified-id="Plots-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Plots</a></span></li></ul></div>

In [37]:
import warnings
import datetime
warnings.filterwarnings("ignore")

# Importing Raw Data

In [14]:
def load_osm_data(country):
    import geopandas as gpd
    import datetime
    from os import listdir
    from os.path import isfile, join
    gdfs_osm=[]

    zipped_shapefile_path = '/mnt/CAS/20240101_foss4g/datasets/in/countries/%s/' %(country)
    onlyfiles = [f for f in listdir(zipped_shapefile_path) if isfile(join(zipped_shapefile_path, f)) if '-latest-' in f]
    print(onlyfiles)
    osm_buildings_stat=[]
    for file in onlyfiles:    
        print(datetime.datetime.now(),'importing osm data for',country,'-',file)
        gdf_osm = gpd.read_file('zip://' + zipped_shapefile_path+file)
        gdf_osm.columns=['osm-id','osm-code','osm-fclass','osm-name','osm-type','geometry']
        print(datetime.datetime.now(),'imported',len(gdf_osm),'rows')
        print(datetime.datetime.now(),gdf_osm['osm-fclass'].unique())
        gdf_osm['source']='osm'
        gdf_osm['dataset']='geofabrik'
        gdf_osm['country']=country
        gdfs_osm.append(gdf_osm)
    import pandas as pd
    gdfs_osm=pd.concat(gdfs_osm)
    return gdfs_osm

In [18]:
def load_eubucco_data(country):
    import warnings
    import datetime
    warnings.filterwarnings("ignore")
    import geopandas as gpd
    import pandas as pd

    zipped_gpkg_path = '/mnt/CAS/20240101_foss4g/datasets/in/countries/%s/eubucco.gpkg.zip' %(country)
    eubucco_buildings_stat=[]
    print(datetime.datetime.now(),'importing Eubucco data for',country)
    gdf_eubucco = gpd.read_file(zipped_gpkg_path)
    print(datetime.datetime.now(),'imported',len(gdf_eubucco),'rows')
    gdf_eubucco['source']='eubucco'
    gdf_eubucco['dataset']='eubucco'
    gdf_eubucco['country']=country
    gdf_eubucco['eubucco-id']=gdf_eubucco['id']
    gdf_eubucco=gdf_eubucco.to_crs(4326)
    return gdf_eubucco

In [19]:
def load_dbsm_data(country,country_extended):
    #https://jeodpp.jrc.ec.europa.eu/ftp/jrc-opendata/DBSM/DBSM_Europe_R2023/per-country/
    import warnings
    import datetime
    warnings.filterwarnings("ignore")
    import geopandas as gpd
    jrc_bdsm = '/mnt/CAS/20240101_foss4g/datasets/in/DBSM_Europe_R2023_v1.zip!DBSM_Europe_R2023_v1/dbsm-v1-%s-merge.gpkg' %(country_extended)
    print(datetime.datetime.now(),'importing JRC-dbsm data for',country,':',country_extended)
    gdf_jrc = gpd.read_file('zip://' + jrc_bdsm)
    gdf_jrc['dataset']='jrc'
    gdf_jrc['country']=country
    gdf_jrc['jrc-id']=gdf_jrc.index
    print(datetime.datetime.now(),'imported',len(gdf_jrc),'rows')
    gdf_jrc=gdf_jrc.to_crs(4326)
    return gdf_jrc

In [20]:
def load_microsoft_data(country):
    import warnings
    import datetime
    warnings.filterwarnings("ignore")
    import geopandas as gpd
    import pandas as pd

    zipped_gpkg_path = '/mnt/CAS/20240101_foss4g/datasets/in/countries/%s/microsoft.gpkg' %(country)
    print(datetime.datetime.now(),'importing Microsoft data for',country)
    gdf_microsoft = gpd.read_file(zipped_gpkg_path)
    print(datetime.datetime.now(),len(gdf_microsoft))
    gdf_microsoft['source']='microsoft'
    gdf_microsoft['dataset']='microsoft'
    gdf_microsoft['country']=country
    gdf_microsoft['microsoft-id']=gdf_microsoft.index
    return gdf_microsoft

In [50]:
def load_nuts(country):
    import geopandas as gpd
    zipped_shapefile_path = '/mnt/CAS/20240101_foss4g/datasets/in/NUTS_RG_01M_2016_4326.shp.zip'
    gdf_lau = gpd.read_file('zip://' + zipped_shapefile_path)
    geo_lau=gdf_lau[(gdf_lau['CNTR_CODE']==country) &(gdf_lau['LEVL_CODE']==3)]
    geo_lau=geo_lau.reset_index()
    del(geo_lau['index'])
    return geo_lau

# Importing enriched data

In [16]:
def load_enriched_data(country,dataset):
    import geopandas as gpd
    import pandas as pd
    import datetime
    print(datetime.datetime.now(),'importing',country,dataset)
    geobuildings=gpd.read_file('/mnt/CAS/20240101_foss4g/datasets/in/countries/%s/%s_enriched.gpkg'%(country,dataset))
    if dataset=='microsoft':
        if not('microsoft-id' in list(geobuildings.columns)):
            geobuildings['microsoft-id']=geobuildings.index
    
    for col in ['area_sqm','num_vertices']:
        column_dtype = geobuildings[col].dtype
        if column_dtype == 'object':
            print(datetime.datetime.now(),"The column %s contains strings. We convert to float" %(col))
            geobuildings[col] = pd.to_numeric(geobuildings[col], errors='coerce')
    
    return geobuildings


In [1]:
def load_enriched_data_dask(country,dataset):
    import dask_geopandas as dgpd
    import pandas as pd
    import datetime
    print(datetime.datetime.now(),'importing',country,dataset)
    geobuildings=dgpd.read_file('/mnt/CAS/20240101_foss4g/datasets/in/countries/%s/%s_enriched.gpkg'%(country,dataset),npartitions=10)
    geobuildings.crs='epsg:4326'
    geobuildings=geobuildings.compute()
    if dataset=='microsoft':
        if not('microsoft-id' in list(geobuildings.columns)):
            geobuildings['microsoft-id']=geobuildings.index
    
    for col in ['area_sqm','num_vertices']:
        column_dtype = geobuildings[col].dtype
        if column_dtype == 'object':
            print(datetime.datetime.now(),"The column %s contains strings. We convert to float" %(col))
            geobuildings[col] = pd.to_numeric(geobuildings[col], errors='coerce')
    
    return geobuildings


# Spatial Join

In [39]:
def building_to_nuts(geo_buildings):
    
    def count_vertices(geometry):
        from shapely.geometry import Polygon, MultiPolygon
        if isinstance(geometry, Polygon):
            return len(geometry.exterior.coords) - 1
        elif isinstance(geometry, MultiPolygon):
            total_vertices = 0
            for polygon in geometry:
                total_vertices += len(polygon.exterior.coords) - 1
            return total_vertices
        else:
            return None  # Handle other geometry types if needed


    import geopandas as gpd
    import pandas as pd
    import warnings
    import datetime
    warnings.filterwarnings("ignore")
    zipped_shapefile_path = '/mnt/CAS/20240101_foss4g/datasets/in/NUTS_RG_01M_2016_4326.shp.zip'
    gdf_lau = gpd.read_file('zip://' + zipped_shapefile_path)
    country=geo_buildings.head(1)['country'].values[0]
    geo_lau=gdf_lau[(gdf_lau['CNTR_CODE']==country) &(gdf_lau['LEVL_CODE']==3)]
    #print(datetime.datetime.now(),' ',len(geo_lau),'nr of lau3 regions per country')
    #print(datetime.datetime.now(),'computing area in sqm with Mercatore projection for',len(geo_buildings),'rows')
    geo_buildings['area']=geo_buildings['geometry'].to_crs({'init': 'epsg:3857'}).map(lambda p: int(p.area ))
    # Apply the function to the 'geometry' column and create a new column 'num_vertices'
    geo_buildings['num_vertices'] = geo_buildings['geometry'].apply(lambda x:count_vertices(x))

    if geo_buildings.crs!='epsg:4326':
        #print(datetime.datetime.now(),'converting to WGS84')
        geo_buildings=geo_buildings.to_crs(4326)
    
    #print(datetime.datetime.now(),'merging buildings and',len(geo_lau),'Lau3 layer')
    result = gpd.sjoin(geo_buildings, geo_lau[['NUTS_ID','URBN_TYPE','NUTS_NAME','geometry']], how='inner', op='intersects')
    result.columns=list(geo_buildings.columns)+['idx_right','nuts_id','urban_type','nuts_name']
    del(result['idx_right'])
    #print(datetime.datetime.now(),'returning results')
    return result

def process_buildings_to_lau_parallel(geo_buildings):
    import warnings
    import datetime
    warnings.filterwarnings("ignore")
    import numpy as np
    import pandas as pd
    import geopandas as gpd
    import multiprocessing as mp
    from pathos.multiprocessing import ProcessingPool as Pool
    cores=mp.cpu_count()
    df_split = np.array_split(geo_buildings, cores, axis=0)
    pool = Pool(cores)
    df_out = np.vstack(pool.map(building_to_nuts, df_split))
    pool.close()
    pool.join()
    pool.clear()

    #merging and aggregating results
    result_parallel=pd.DataFrame(df_out)
    result_parallel.columns=list(geo_buildings.columns)+['area_sqm','num_vertices','nuts_id','urban_type','nuts_name']
    result_parallel=gpd.GeoDataFrame(result_parallel,geometry='geometry')
    return result_parallel

# Aggregation

In [2]:
def aggregation(geobuildings):
    import pandas as pd
    functions = {
    'geometry': 'count',
    'area_sqm': 'sum',
    'num_vertices':'sum'
    }

    # Apply different functions to group by 'Group' column
    buildings_stat = pd.DataFrame(geobuildings.groupby(['dataset','source','nuts_id','nuts_name','urban_type','country']).agg(functions)).reset_index()
    buildings_stat = buildings_stat.rename(columns={'geometry': 'nr_buildings'})

    print(datetime.datetime.now(),'total number of regions computed:',len(buildings_stat))
    return buildings_stat


# Plots

In [61]:
def plot_country_geometries(country):
    import geopandas as gpd
    import matplotlib.pyplot as plt
    import seaborn as sns
    sns.set_style("white")
    from matplotlib.colors import ListedColormap
    from matplotlib.patches import Patch

    # Create a custom legend handler for the intersection
    class YellowBoxHandler(Patch):
        def __init__(self, color='yellow', alpha=0.5, **kwargs):
            super().__init__(color=color, alpha=alpha, **kwargs)

    # Importing country
    geo_lau = load_nuts(country)
    geo_lau['URBN_TYPE'] = [str(val) for val in geo_lau['URBN_TYPE'].values]

    #computing total area per urban type
    geo_lau['area']=geo_lau['geometry'].to_crs({'init': 'epsg:3857'}).map(lambda p: int(p.area ))

    # Define custom colors for each urban type
    custom_colors = {
        '1': 'darkred',  # Dark red for urban type 1
        '2': 'blue',     # Blue for urban type 2
        '3': 'green'     # Green for urban type 3
    }
    geo_lau['colors']=[custom_colors[val] for val in geo_lau['URBN_TYPE'].values]
    # Set Seaborn style

    # Set figure size
    plt.figure(figsize=(12, 10))

    #statistic about area
    import numpy as np
    area_stat=pd.DataFrame(geo_lau.groupby(['URBN_TYPE'])['area'].sum()).to_dict(orient='index')
    sum_area=0.0
    for ut in area_stat:
        if ut!='3':
            a=np.round(area_stat[ut]['area']/geo_lau['area'].sum()*100,2)
            area_stat[ut]=a
            sum_area+=a
        else:
            area_stat[ut]=np.round(100-sum_area,2)
    for key in ['1','2','3']:
        if not(key in area_stat):
            area_stat[key]=0

    # Plot GeoDataFrame with the created ListedColormap
    geo_lau.plot(column='URBN_TYPE', legend=True, color=geo_lau['colors'], edgecolor="none",alpha=0.8)

    # Add title and labels if needed
    plt.title('%s by Degree of Urbanization' % (country))
    plt.xlabel('Longitude')
    plt.ylabel('Latitude')
    #Cities; Towns and suburbs; Rural areas
    legend_labels = ['Urban ('+str(area_stat['1'])+' %)', 'Semi Urban ('+str(area_stat['2'])+' %)', 'Rural ('+str(area_stat['3'])+' %)']

    # Create legend handles
    handles = [
        YellowBoxHandler(color='darkred', alpha=0.8),
        YellowBoxHandler(color='blue', alpha=0.8),
        YellowBoxHandler(color='green', alpha=0.8)
    ]

    # Create the legend
    plt.legend(handles=handles, labels=legend_labels,bbox_to_anchor=(0.5, -0.18), ncol=2,loc='upper center')

    plt.subplots_adjust(bottom=0.26)

    # Show the plot
    plt.savefig('/mnt/CAS/20240101_foss4g/plots/countries/%s.png' %(country),dpi=300)